In [2]:
!nvidia-smi

Tue Jan 30 18:53:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           On  | 00000000:06:00.0 Off |                    0 |
| N/A   39C    P0              60W / 300W |   1543MiB / 16384MiB |      0%   E. Process |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
os.environ['CUDA_LAUNCH_BLOCKING']="1"
import torch

torch.tensor([3]).to("cuda")

tensor([3], device='cuda:0')

In [7]:
device = torch.device('cuda')
torch.randn(10).to(device)

tensor([ 0.4451,  1.2140, -1.1644,  0.5476,  0.5005,  2.6480, -1.0846, -0.8021,
         1.6652, -0.3085], device='cuda:0')

In [ ]:
import sys
# sys.path.append('./..')
# sys.path.append('./img2img')
import os
os.environ['CUDA_VISIBLE_DEVICES']="1"
os.environ['CUDA_LAUNCH_BLOCKING']="1"
import json

import torch
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision.models.resnet import resnet50
import scipy

from architectures import get_architecture
from datasets_utils import get_dataset, DATASETS, get_num_classes, get_normalize_layer
from src.numerical_methods import *
from src.certification_utils import *
from src.smoothing_and_attacks import *
from src.utils import *



NUM_IMAGES_FOR_TEST = 500

sigma_b = None
sigma_c = None
sigma_tr = None
sigma_gamma = None
sigma_blur = None
xi_tss = scipy.stats.norm.ppf

rng = jax.random.PRNGKey(33)
rng, key = jax.random.split(rng)
ns = 10_000
normal_samples = jax.random.normal(key, [100_000])


parser = argparse.ArgumentParser(description='Certify many examples')
parser.add_argument("--dataset",
                    default='cifar10',choices=DATASETS, help="which dataset")
parser.add_argument("--base_classifier", type=str,
                    default=None,
                    help="path to saved pytorch model of base classifier")
# parser.add_argument("sigma", type=float, help="noise hyperparameter")
parser.add_argument("--outfile", type=str,
                    default='./logs/certify_results',
                    help="output file")
parser.add_argument("--batch", type=int, default=512, help="batch size")
parser.add_argument("--skip", type=int, default=30
                    , help="how many examples to skip")
parser.add_argument("--max", type=int, default=-1, help="stop after this many examples")
parser.add_argument("--split", choices=["train", "test"], default="test", help="train or test set")
parser.add_argument("--n0", type=int, default=200)
parser.add_argument("--maxn", type=int, default=1000, help="number of samples to use")
parser.add_argument("--alpha", type=float, default=1e-3, help="failure probability")
parser.add_argument('--corrupt',type=str,default=['none','gaussian_blur','motion_blur','zoom_blur','rotate','translate','contrast','pixelate','jpeg',][1],
                    help=' The corruption type for training')
parser.add_argument('--add_noise',type=float, default=0.0)
parser.add_argument('--noise_dst',default=["none","gaussian","exp","uniform","folded_gaussian"][2],type=str)
parser.add_argument('--noise_sd', default=0.8, type=float,
                    help="standard deviation of Gaussian noise for data augmentation")
parser.add_argument('--partial_min',default=0.0, type=float,
                    help = "Minimal of certify range")
parser.add_argument('--adaptive',default=True, type=bool,
                    help = "whether to stop certification procedure earlier")
parser.add_argument('--partial_max',default=1.0, type=float,
                    help = "Maximal of certify range")
parser.add_argument('--arch', type=str, default=["edsr","unet","runet"][2])
args = parser.parse_args([])


def make_test_dataset(args):
    test_dataset = get_dataset(args.dataset, "test")
    pin_memory = (args.dataset == "imagenet")
    np.random.seed(42)
    idxes = np.random.choice(len(test_dataset), NUM_IMAGES_FOR_TEST, replace=False)
    
    ourdataset = make_our_dataset_v2(test_dataset, idxes)
    ourdataloader = DataLoader(ourdataset, shuffle=False, batch_size=1,
                         num_workers=6, pin_memory=False)
    return ourdataset, ourdataloader


def construct_bounds(ns, b_zero, x0, d, betas_list, type_of_transform, sigmas=[None, None, None, None, None]):
    shape = [b.shape[0] for b in betas_list]
    shape = tuple(shape)
    betas = jnp.asarray(list(map(jnp.array, itertools.product(*betas_list))))
#     sigma_b, sigma_c, sigma_tr, sigma_gamma, sigma_blur = sigmas
#     gamma = construct_gamma(sigma_b=sigma_b, sigma_c=sigma_c, sigma_tr=sigma_tr, sigma_gamma=sigma_gamma, sigma_blur=sigma_blur)
    gamma = construct_gamma(*sigmas)
    bounds, p, g = compute_normed_bounds(compute_bound, x0, gamma, b_zero, betas, key, ns, d, type_of_transform)
    x, xi = pxi_to_xi(p)
#     for b in bounds[:1000]:
#         plt.plot(b)
#     plt.show()

#     plt.plot(g)
#     plt.show()

#     plt.plot(p)
#     plt.show()
    z = csaps(betas_list, g.reshape(shape))
    
    hg = []

    for beta in tqdm(betas):
        hat_g = g_to_hat_g(z, beta, b_zero)
        hg.append(hat_g)

    hat_g = jnp.asarray(hg)

    hatg_int = csaps(betas_list, hat_g.reshape(shape)) #
    return xi, hatg_int



def calculate_general(args):
    n0 = args.n0
    maxn = args.maxn
    adaptive = False
    alpha = args.alpha
    
    device = torch.device(args.device)
    model = get_architecture(arch=args.arch, dataset=args.dataset, device=device)
    checkpoint = torch.load(args.base_classifier, map_location=device)
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    model = model.to(device)


    
    dataset, dataloader = make_test_dataset(args)
    
    base_classifier.to(device)
    base_classifier.eval()

    corruptor = Corruption(args, co_type=args.corrupt,add_noise=args.add_noise,noise_sd=args.noise_sd,distribution=args.noise_dst)

    # create the smooothed classifier g
    smoothed_classifier = TSmooth(base_classifier, None, corruptor, get_num_classes(args.dataset),args.noise_dst,args.noise_sd, args.add_noise)

    # prepare output file
    filename = args.outfile+'_'+args.dataset+'_'+args.corrupt+'_'+str(args.noise_sd) +"_" +str(args.partial_max)
    f = open(filename+'_running', 'w')
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime", file=f, flush=True)
    print("idx\tlabel\tpredict\tradius\tgood\tcorrect\ttime")

    tot, tot_good, tot_correct = 0, 0, 0

    # for gaussian smooth
    attack_radius = args.partial_max
    for i in tqdm(range(len(dataset))):
        (x, label) = dataset[i]

        before_time = time()
        x = x.to(device)
        prediction, radius = smoothed_classifier.certify(x, args.N0, args.N, args.alpha, args.batch)


        correct = (prediction == label).item()
        cond1 = radius * args.noise_sd > args.partial_max
        good = (radius * args.noise_sd > args.partial_max)&correct

        tot, tot_good, tot_correct = tot+1, tot_good+good, tot_correct+correct
        after_time = time()
        time_elapsed = str(datetime.timedelta(seconds=(after_time - before_time)))
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed), file=f, flush=True)
        print("{}\t{}\t{}\t{:.5f}\t{}\t{}\t{}".format(i, label, prediction, radius, good, correct, time_elapsed))

    f.close()

    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot))

    f = open(filename+'_total_result', 'w')
    print("Total {} Certified {} Certified Acc {} Test Acc {}".format(tot, tot_good, tot_good/tot, tot_correct/tot), file=f, flush=True)
    f.close()

    
@click.command()
@click.argument("config_path", type=click.Path(exists=True))
def main(config_path):
    with open(config_path, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
        
    args.corrupt = config["corrupt"]
    args.noise_sd = config["noise_sd"]
    args.noise_dst = config["noise_dst"]
    args.partial_max = config["partial_max"]
    args.dataset = config["dataset"]
    args.base_classifier = config["base_classifier"]
    args.device = config["device"]
    args.arch = config["arch"]
    
#     if config["gpu"]:
#         os.environ['CUDA_VISIBLE_DEVICES']=str(config["gpu"])
#         os.environ['CUDA_LAUNCH_BLOCKING']="1"
        


    calculate_gsmooth(args)

if __name__ == "__main__":
    main()

In [4]:
!ls

README.md	  dataset_cache		 requirements_old.txt
__init__.py	  datasets_utils.py	 results
__pycache__	  gsmooth		 src
architectures.py  new_results		 train.py
archs		  new_results.tar.gz	 train_utils.py
checkpoints	  notebooks		 trying_maker_file.ipynb
checkpoints.tar   requirements.txt	 tss_weights
configs		  requirements_full.txt  wandb


In [11]:
!pip install python-box

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for python-box from https://files.pythonhosted.org/packages/88/cf/c75636ecfc9185630d6359fbb802b6a137a6cc763e0af6597a96b54bd343/python_box-7.1.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 14.8 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [58]:
import yaml
from box import Box
config_path = "configs/cb/imagenet.yaml"
with open(config_path, "r") as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [59]:
config = Box.from_yaml(filename=config_path, Loader=yaml.FullLoader) 

In [60]:
len(config.betas_estimation)

2

In [61]:
config

Box({'transform': 'cb', 'dimenshion': 2, 'dataset': 'imagenet', 'base_classifier': '../../checkpoints/brightness/checkpoint.pth.tar', 'device': 'cuda', 'arch': 'resnet50', 'gpu': 5, 'sigmas': {'sigma_b': 0.3, 'sigma_c': 0.3, 'sigma_tr': None, 'sigma_blur': None, 'sigma_gamma': None}, 'betas_estimation': {'beta0': {'left': 0.5, 'right': 1.5, 'db': 21}, 'beta1': {'left': -0.5, 'right': 0.5, 'db': 23}}, 'betas_certification': {'beta0': {'left': 1.4, 'right': 1.4, 'db': 35}, 'beta1': {'left': -0.4, 'right': 0.4, 'db': 41}}})